<a href="https://colab.research.google.com/github/seawavve/NLU_STS/blob/main/NLU_STS.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# NLU - 문장 유사도 계산 (STS)
11팀 조한희 강지호 윤진헌 김예림 김성우 신원지

## SetUp

In [1]:
! git clone https://github.com/seawavve/NLU_STS
%cd NLU_STS

Cloning into 'NLU_STS'...
remote: Enumerating objects: 81, done.
remote: Counting objects: 100% (81/81), done.
remote: Compressing objects: 100% (73/73), done.
remote: Total 81 (delta 28), reused 16 (delta 3), pack-reused 0
Unpacking objects: 100% (81/81), done.
/content/NLU_STS


In [2]:
# sentence trainsformer 설치
!pip install -U sentence-transformers
!pip install konlpy
!pip install wandb --upgrade

     |████████████████████████████████| 79 kB 5.1 MB/s 
     |████████████████████████████████| 3.8 MB 18.8 MB/s 
     |████████████████████████████████| 1.2 MB 48.2 MB/s 
     |████████████████████████████████| 67 kB 3.0 MB/s 
     |████████████████████████████████| 895 kB 72.6 MB/s 
     |████████████████████████████████| 596 kB 53.5 MB/s 
     |████████████████████████████████| 6.5 MB 54.6 MB/s 
  Created wheel for sentence-transformers: filename=sentence_transformers-2.2.0-py3-none-any.whl size=120747 sha256=85591893f9b55e661a6f96046c476b13940b8c5bbc0b7fe52e1016240f0223ad
  Stored in directory: /root/.cache/pip/wheels/83/c0/df/b6873ab7aac3f2465aa9144b6b4c41c4391cfecc027c8b07e7
Successfully built sentence-transformers
  Attempting uninstall: pyyaml
    Found existing installation: PyYAML 3.13
    Uninstalling PyYAML-3.13:
      Successfully uninstalled PyYAML-3.13
     |████████████████████████████████| 19.4 MB 12.3 MB/s 
     |████████████████████████████████| 448 kB 83.7 MB/s 
   

In [3]:
import os
import sys
import pandas as pd
import numpy as np 
import random
import json
import math
import logging
from datetime import datetime

import torch
from torch.utils.data import Dataset, DataLoader, RandomSampler, SequentialSampler, random_split
from sentence_transformers import models, LoggingHandler, util ,losses
from sentence_transformers.readers import InputExample
from sentence_transformers.evaluation import EmbeddingSimilarityEvaluator
import wandb

from model.logging_model import *

## Load Processed & Augumented Data

In [4]:
import sys
from sklearn.model_selection import train_test_split
sys.path.append('./data_preprocessing')
import preprocessing as pp

path = './data/'    
n = 2
_pp = pp.data_preprosessing(path, n)
_train, _test = _pp.load_origin_file()
_train, _valid = train_test_split(_train, test_size=0.1, random_state=42) #***별도로 진행해서 train, valid 만들어도 됨***
train, valid, test = _pp.load_all(_train, _valid, _test)

Length of original DF : 11661
Length of original dev : 519

************** cleaning : train **************


Cleaning start


column name : sentence1

Remove chiness charecter start


100%|██████████| 10494/10494 [00:00<00:00, 448830.65it/s]


Remove english start


100%|██████████| 10494/10494 [00:00<00:00, 536265.05it/s]


Remove special charecter start


...

Remove repeated spacing start


100%|██████████| 41976/41976 [00:00<00:00, 457750.78it/s]


Remove repeated spacing start


100%|██████████| 41976/41976 [00:00<00:00, 256727.47it/s]



************** data augmentation train length : 41279 **************


************** concat : train **************


************** concat train length : 60684 **************


************** saving files : train, valid, test **************


************** train length : 60684, valid length : 1167, test length : 519 **************



In [5]:
# 미리 전처리된 파일 사용
def str_to_json(x):
    x = x.replace("'", '"')
    x = json.loads(x)

    return x

train_path = "./data/STS_train.csv"
valid_path = "./data/STS_valid.csv"
test_path = "./data/STS_test.csv"
train = pd.read_csv(train_path)
valid = pd.read_csv(valid_path)
test = pd.read_csv(test_path)

train['labels'] = train['labels'].apply(str_to_json)
valid['labels'] = valid['labels'].apply(str_to_json)
test['labels'] = test['labels'].apply(str_to_json)

print(f'Length of Train : {len(train)}') 
print(f"Length of Valid : {len(valid)}")
print(f'Length of Test : {len(test)}') 

Length of Train : 60678
Length of Valid : 1167
Length of Test : 519


In [6]:
# 최종 데이터 head
display(train.head)
display(valid.head)
display(test.head)

<bound method NDFrame.head of                                                sentence1  \
0          다만 1월달에 공사소리때문에 시끄러웠다는 후기를봤는데 여전히 공사를 하는듯합니다.   
1                            경쟁력 위원회는 이날 6건의 협력사업도 승인했다.   
2                       다만 공용 거실이 없어 2명이 지내기에는 좁은 듯 합니다.   
3                             여행기간 내내 잠을 잘 자서 힐링이 잘 되었어요   
4                      대신 제일 꼭대기층이어서 캐리어 가져가기 좀 힘들긴 합니다.   
...                                                  ...   
60673                     호스트와 연락이 바로 된점과 침구가 제일 맘에 들었어요   
60674                       위치도 시설도 호스트분도 부족함 없이 완벽했습니다.   
60675                       위치도 시설도 호스트분도 부족함 없이 완벽했습니다.   
60676  올해부터 부산 수영구 대구 남구 인천 연수구 등 5개 지역의 문화향유사업이 통합 지...   
60677  올해부터 부산 수영구 대구 남구 인천 연수구 등 5개 지역의 문화향유사업이 통합 지...   

                                               sentence2  \
0                   다만 1인실용이기 때문에 다소 좁지만 혼자서 이용하긴 충분합니다.   
1                                결함신고와 리콜현황의 통계기능도 강화했다.   
2                하지만 공공 거실이 없어서 두 사람이 머물기에는 너무 작은 것 같아요.   
3        

<bound method NDFrame.head of                                               sentence1  \
0                                    우산 두고 오지 않았으면 좋겠어.   
1                                 다만 밤에 돌아다니기엔 조금 무섭습니다   
2     중소벤처기업부 등 17개 부처가 마련한 방안은 코로나19 피해기업 지원과 농업 신산...   
3      2020 부산국제광고제 와 관련된 더욱 자세한 내용은 부산국제광고제 누리집 .. ...   
4                          모던한 느낌으로 모든 사람들이 좋아할만한 집입니다.   
...                                                 ...   
1162                     치과를 예약하고 싶은데 다음주 중 되는 시간이 얹에요    
1163  국토부는 고령가구에 대해서는 2025년까지 무장애 설계가 적용된 공공임대 8만호를 ...   
1164                             친구들과 지내기 정말 좋은 장소였습니다.   
1165                   믹서기 말고 가스레인지로 요리할 수 있도록 협조 바랍니다.   
1166                                    냉장고 문 열려고 시도하지마   

                                              sentence2  \
0                                     우산 두고 오지 않기로 했잖아    
1                                     밤에 돌아다니기가 좀 무서워요.   
2             문체부 등 각 부처 지원 사업 안내 코로나19 피해 상담 등 통합창구 개설   
3     부산국제광고제에 대한 자세한 내용은

<bound method NDFrame.head of                                              sentence1  \
0                             무엇보다도 호스트분들이 너무 친절하셨습니다.   
1                               주요 관광지 모두 걸어서 이동가능합니다.   
2    학생들의 균형 있는 영어능력을 향상시킬 수 있는 학교 수업을 유도하기 위해 2018...   
3                             다만 도로와 인접해서 거리의 소음이 들려요.   
4                  형이 다시 캐나다 들어가야 하니 가족모임 일정은 바꾸지 마세요.   
..                                                 ...   
514     문체부는 이를 연차적으로 확대 시행해 학교운동부와 스포츠클럽 간의 연계를 강화한다.   
515                        일단 정확한 정보와 빠른 답변이 정말 좋았습니다.   
516                         게스트에 대한 배려가 묻어나는 시설들이었습니다.   
517                          밤하늘을 배경으로 비대면 드론쇼 도 펼쳐진다.   
518                      여느 포르투갈의 비앤비와 같이 엘리베이터는 없습니다.   

                                             sentence2  \
0                               무엇보다도 호스트들은 매우 친절했습니다.   
1                          위치는 피렌체 중심가까지 걸어서 이동 가능합니다.   
2    영어 영역의 경우 학생들이 한글 해석본을 암기하는 문제를 해소하기 위해 2016학년...   
3                    하지만 길과 가깝기 때문에 거리의 소

In [7]:
from sentence_transformers.readers import InputExample

class STSDataset(Dataset):
  def __init__(self, df:pd.DataFrame):
      self.texts = list(zip(df['sentence1'].tolist(),df['sentence2'].tolist()))
      self.texts = [list(x) for x in self.texts]
      self.labels = df['normalized'].tolist()
      self.input_examples = self.make_sts_input(self.texts,self.labels)

  def __len__(self):
      return len(self.input_examples)

  def __getitem__(self, index):
      return self.input_examples[index]

  def make_sts_input(self,texts,labels):
      input_examples =[]
      for text,label in zip(texts,labels):
          input = InputExample(texts = text,label = label)
          input_examples.append(input)

      return input_examples 
       

In [8]:
train_dataset = STSDataset(train)
valid_dataset = STSDataset(valid)
test_set = STSDataset(test)

In [9]:
print(f"Train Dataset len: {len(train_dataset)}")
print(f"Train Dataset 1st element: {train_dataset[0]}\n")

print(f"Valid Dataset len: {len(valid_dataset)}")
print(f"Valid Dataset 1st element: {valid_dataset[0]}\n")

print(f"Test Dataset len: {len(test_set)}")
print(f"Test Dataset 1st element:{test_set[0]}")

Train Dataset len: 60678
Train Dataset 1st element: <InputExample> label: 0.04, texts: 다만 1월달에 공사소리때문에 시끄러웠다는 후기를봤는데 여전히 공사를 하는듯합니다.; 다만 1인실용이기 때문에 다소 좁지만 혼자서 이용하긴 충분합니다.

Valid Dataset len: 1167
Valid Dataset 1st element: <InputExample> label: 0.6, texts: 우산 두고 오지 않았으면 좋겠어.; 우산 두고 오지 않기로 했잖아 

Test Dataset len: 519
Test Dataset 1st element:<InputExample> label: 0.98, texts: 무엇보다도 호스트분들이 너무 친절하셨습니다.; 무엇보다도 호스트들은 매우 친절했습니다.


In [10]:
def ret_dataloader(train_dataset,valid_dataset,batch_size : int ):
    _batch_size = batch_size
    train_dataloader = DataLoader(train_dataset,shuffle=True , batch_size = _batch_size)
    valid_evaluator = EmbeddingSimilarityEvaluator.from_input_examples(valid_dataset,name="sts-dev")
    
    return train_dataloader,valid_evaluator

In [11]:
test_evaluator = EmbeddingSimilarityEvaluator.from_input_examples(test_set,name='sts_test')

## Model

In [12]:
model_path = './model'
sys.path.append(model_path)

### Parameter Tuning

In [13]:
import wandb
wandb.login()

# Optional: log both gradients and parameters
%env WANDB_WATCH=all

<IPython.core.display.Javascript object>

wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc


env: WANDB_WATCH=all


In [14]:
project_name = "klue_sts_roberta_b"
wandb.init(project=project_name)

wandb: Currently logged in as: seawavve (use `wandb login --relogin` to force relogin)


In [15]:
# hypermar 설정을 위한 sweep

sweep_config = {
    'method': 'random'
    }

metric = {
    'name': 'loss',
    'goal': 'minimize'   
    }

parameters_dict = {
    # Optimizer: adamw
    'epochs' : {
        'value': 4
    },
    'optimizer_params': {
        'values' : [
                {'lr':1e-4},
                {'lr':3e-4},
                {'lr':5e-5}
                ]
    },
    'weight_decay':{
        'distribution': 'uniform',
        'min': 0.01,
        'max': 0.1
    },
    'batch_size' :{
        'values': [16,32,64]
      }
    }




sweep_config['metric'] = metric
sweep_config['parameters'] = parameters_dict

In [16]:
import pprint
pprint.pprint(sweep_config)

{'method': 'random',
 'metric': {'goal': 'minimize', 'name': 'loss'},
 'parameters': {'batch_size': {'values': [16, 32, 64]},
                'epochs': {'value': 4},
                'optimizer_params': {'values': [{'lr': 0.0001},
                                                {'lr': 0.0003},
                                                {'lr': 5e-05}]},
                'weight_decay': {'distribution': 'uniform',
                                 'max': 0.1,
                                 'min': 0.01}}}


In [17]:
sweep_id = wandb.sweep(sweep_config, project=project_name)

Create sweep with ID: qp8svt7a
Sweep URL: https://wandb.ai/seawavve/klue_sts_roberta_b/sweeps/qp8svt7a


In [18]:
# Train 생성 함수
def ret_model(model_name):
    # Load Embedding Model 
    embedding_model = models.Transformer(
            model_name_or_path=model_name, 
            max_seq_length=256,
            do_lower_case=False
            )
    # Only use Mean Pooling -> Pooling all token embedding vectors of sentence. 
    """
    Using pooling, it generates from a variable sized sentence a fixed sized sentence embedding. This layer also allows to use the CLS token if it is returned by the underlying word embedding model.
    You can concatenate multiple poolings together
    """   
    pooling_model = models.Pooling(
        embedding_model.get_word_embedding_dimension(), 
        pooling_mode_mean_tokens=True,
        pooling_mode_cls_token=False,
        pooling_mode_max_tokens=False,)
    
    model = SentenceTransformer(modules=[embedding_model, pooling_model])

    return model


In [19]:
from pytz import timezone
from datetime import datetime

def set_model_path(save_path,model_name:str):
    today = datetime.now(timezone('Asia/Seoul'))
    save_path = save_path
    model_save_path = save_path+'/training_sts-'+model_name.replace("/", "-")+'-'+today.strftime("%Y-%m-%d_%H-%M-%S")

    return model_save_path

In [20]:
model_name = "Huffon/sentence-klue-roberta-base"
save_path = './model_files'
# save_path = '/content/drive/MyDrive/Colab Notebooks/NLP_torch/기업과제/STS/models'

def train(config=None,model = None):
    # Initialize a new wandb run
    with wandb.init(config=config):
        # If called by wandb.agent, as below,
        # this config will be set by Sweep Controller
        config = wandb.config
        model = ret_model(model_name)

        # set loss
        train_loss = losses.CosineSimilarityLoss(model=model) 
        model_save_path = set_model_path(save_path,model_name=model_name)

        # set batch_size
        train_batch_size = config.batch_size

        train_dataloader , valid_evaluator = ret_dataloader(train_dataset,valid_dataset,batch_size = config.batch_size)
        print('train data loaded')
        
        train_loss = losses.CosineSimilarityLoss(model=model)
        warmup_steps = math.ceil(len(train_dataset) * config.epochs / train_batch_size * 0.1) #10% of train data for warm-up
        model.fit(
            train_objectives=[(train_dataloader, train_loss)],
            evaluator=valid_evaluator,
            epochs=config.epochs,
            optimizer_params = config.optimizer_params,
            weight_decay = config.weight_decay,
            evaluation_steps=int(len(train_dataloader)*0.1),
            warmup_steps=warmup_steps,
            output_path= model_save_path,
            show_progress_bar = True
        )
   

In [21]:
wandb.agent(sweep_id, train, count=5)

wandb: WARNING Calling wandb.login() after wandb.init() has no effect.
wandb: Agent Starting Run: fu0bmuji with config:
wandb: 	batch_size: 64
wandb: 	epochs: 4
wandb: 	optimizer_params: {'lr': 5e-05}
wandb: 	weight_decay: 0.014292202998185462


Downloading:   0%|          | 0.00/744 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/422M [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/558 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/243k [00:00<?, ?B/s]

...

/usr/local/lib/python3.7/dist-packages/transformers/optimization.py:309: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  FutureWarning,


Epoch:   0%|          | 0/4 [00:00<?, ?it/s]

Iteration:   0%|          | 0/949 [00:00<?, ?it/s]


################ F1 Score 계산 ################
              precision    recall  f1-score   support

           0       1.00      0.93      0.96       596
           1       0.93      1.00      0.97       571

    accuracy                           0.96      1167
   macro avg       0.97      0.97      0.96      1167
weighted avg       0.97      0.96      0.96      1167


################ F1 Score 계산 ################
              precision    recall  f1-score   support

           0       1.00      0.93      0.96       596
           1       0.93      1.00      0.96       571

    accuracy                           0.96      1167
   macro avg       0.97      0.96      0.96      1167
weighted avg       0.97      0.96      0.96      1167


################ F1 Score 계산 ################
              precision    recall  f1-score   support

           0       1.00      0.94      0.97       596
           1       0.94      1.00      0.97       571

    accuracy                           0.

wandb: Ctrl + C detected. Stopping sweep.


In [ ]:
################## NOT IMPLEMENTED ################

In [22]:
# get path in model save
all_model_path = [x for x in os.listdir(save_path) if x.startswith("sts-"+model_name)]

def find_best_model(all_model_path,test_set):
    for m in all_model_path:
        model = SentenceTransformer("../model_final/"+m)
        sentences1 = []
        sentences2 = []
        scores = []
        for sample in test_set:
            sentences1.append(sample.texts[0])
            sentences2.append(sample.texts[1])
            scores.append(sample.label)
        
        corpus_embeddings = model.encode(sentences1, convert_to_tensor=True)
        query_embeddings = model.encode(sentences2, convert_to_tensor=True)
        
        pred = []
        
        for i in range(len(test_set)):
            score = torch.nn.functional.cosine_similarity(corpus_embeddings[i], query_embeddings[i])
            score = score.cpu().detach().numpy()
            pred.append(score)
            
        pred = np.array(pred) # 모델 예측값
        y_pred = np.where(pred>=0.6, 1, 0) # normalised score 기준으로 threshold를 0.6 으로 둠
        scores = np.array(pred)
        y_label = np.where(scores >= 0.6, 1, 0)
        print(classification_report(y_label, y_pred))

## Serving
[해당 Repository](https://github.com/seawavve/STS_serving)를 참고 바랍니다.